In [18]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import re
import tensorflow.keras as krs

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    krs.backend.clear_session()
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Preprocess the data

In [19]:
# TODO (maybe) : split on songs

file_path = "data/taylor_swift_lyrics.csv"

# Read the dataset from csv
df = pd.read_csv(filepath_or_buffer=file_path,
                encoding = "ISO-8859-1",
                sep=";")

# Extract the lyrics column
lyrics = df['lyric']

# Remove all special characters and add the words to a list
all_chars = []
for line in lyrics:
    for char in list(line):
        c = re.sub('[^A-Za-z0-9\s]+', '', char)
        all_chars.append(c)
    all_chars.append("\n")

# Get all unique characters from the dataset
unique_chars = list(set(all_chars))
        
# Create translation tables from char -> ind and ind -> char for one-hot encoding
chars_to_ind = dict((c, i) for i, c in enumerate(unique_chars))
ind_to_chars = dict((i, c) for i, c in enumerate(unique_chars))

N = len(all_chars)
d = len(unique_chars)

print("All chars: ", N)
print("Unique chars: ", d)


All chars:  173542
Unique chars:  63


## One-Hot encoding of the lyrics data

In [ ]:
X = np.zeros((N, d))
for i,char in enumerate(all_chars):
    X[i][chars_to_ind[char]] = 1

Let's train an RNN to classify MNIST images. We will treat each image as a sequence of 28 rows of 28 pixels each (since each MNIST image is 28×28 pixels). Assume we will use cells of 100 recurrent neurons, plus a fully connected layer containing 10 neurons connected to the output of the last time step, followed by a softmax layer.

In [ ]:
# multi-layer RNN network for mnist
reset_graph()

seq_length = 100

model = krs.Sequential()
model.add(krs.layers.LSTM(d, input_shape=(seq_length, d)))
model.add(krs.layers.Dropout(0.2))
model.add(krs.layers.Dense(d, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

nb_epoch=10
batch_size=200
nr_batches = int(N / batch_size)

for e in range(nb_epoch):
    print("epoch %d" % e)
    for b in range(nr_batches):
        X_batch=np.zeros((batch_size, seq_length, d))
        Y_batch=np.zeros((batch_size, d))
        for i in range(batch_size):
            X_batch[i] = X[(b*batch_size) + i:(b*batch_size) + i + seq_length]
            Y_batch[i] = X[(b*batch_size) + i + seq_length]
            model.fit(X_batch, Y_batch, batch_size=50, epochs=1, verbose=0)

########################################
# executing the model
########################################


epoch 0


We build the above network again for the MNIST dataset, but this time we use LTSM instead of regular RNN.

In [ ]:
# building an LSTM RNN
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
n_layers = 3
learning_rate = 0.001
n_epochs = 10
batch_size = 150

########################################
# loading dataset
########################################
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels.astype("int")

########################################
# defining variables and placeholders
########################################
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

########################################
# building the model
########################################
with tf.name_scope("rnn"):
    lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons) for layer in range(n_layers)]
    multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
    top_layer_h_state = states[-1][1]
    logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")

########################################
# defining the loss function
########################################
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
########################################
# training the model
########################################
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
########################################
# defining the evaluation metrics
########################################
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
########################################
# executing the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)